___

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import math
import time

# 폰트 색상 지정
blue = (255, 0, 0)
green= (0, 255, 0)
red= (0, 0, 255)
white= (255, 255, 255) 
# 폰트 지정
font =  cv2.FONT_HERSHEY_PLAIN

#from scipy.spatial import distance as dist
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

# right eyes indices
RIGHT_EYE=[33, 160, 158, 133, 153, 144] 

# Left eyes indices 
LEFT_EYE =[263, 387, 385, 362, 380, 373] 

def euclideanDistance(point, point1):
    x, y = point
    x1, y1 = point1
    distance = math.sqrt((x1 - x)**2 + (y1 - y)**2)
    return distance

# return EAR
def eye_aspect_ratio(image, landmarks, right_indices, left_indices):
    rh_right = landmarks[right_indices[0]][:2]
    rh_left = landmarks[right_indices[3]][:2]
    rv_top1 = landmarks[right_indices[2]][:2]
    rv_top2 = landmarks[right_indices[1]][:2]
    rv_bottom1 = landmarks[right_indices[4]][:2]
    rv_bottom2 = landmarks[right_indices[5]][:2]
    
    lh_right = landmarks[left_indices[0]][:2]
    lh_left = landmarks[left_indices[3]][:2]
    lv_top1 = landmarks[left_indices[2]][:2]
    lv_top2 = landmarks[left_indices[1]][:2]
    lv_bottom1 = landmarks[left_indices[4]][:2]
    lv_bottom2 = landmarks[left_indices[5]][:2]
    
    rhDistance = euclideanDistance(rh_right, rh_left)
    rvDistance1 = euclideanDistance(rv_top1, rv_bottom1)
    rvDistance2 = euclideanDistance(rv_top2, rv_bottom2)
    rvDistance = rvDistance1 + rvDistance2
    
    lvDistance1 = euclideanDistance(lv_top1, lv_bottom1)
    lvDistance2 = euclideanDistance(lv_top2, lv_bottom2)
    lhDistance = euclideanDistance(lh_right, lh_left)
    lvDistance = lvDistance1 + lvDistance2

    reRatio = rvDistance/rhDistance/2
    leRatio = lvDistance/lhDistance/2

    ear = (reRatio+leRatio)/2
    
    return reRatio

drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)

prevTime = time.time()
totalTime = 0
currTime = 0

open_ear = []
close_ear = []

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
        while cap.isOpened():
            success, image = cap.read()

            if not success:
                print("웹캠 인식 불가")
                continue

            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # image로부터 랜드마크 반환
            results = face_mesh.process(image)
            
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            cv2.putText(image, "enter q and open eyes 5 sec", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, red, 3)
            cv2.imshow('hello', image)
            
            if cv2.waitKey(5) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
            # 반환된 랜드마크를 image에 덧씌움
                with mp_face_mesh.FaceMesh(
                max_num_faces=1,
                refine_landmarks=True,
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5) as face_mesh:
                    prevTime = time.time()
                    totalTime = 0
                    currTime = 0
                    while cap.isOpened():
                        success, image = cap.read()
                        image.flags.writeable = False
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                        # image로부터 랜드마크 반환
                        results = face_mesh.process(image)

                        image.flags.writeable = True
                        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                        if results.multi_face_landmarks:
                            for face_landmarks in results.multi_face_landmarks:
                                mypose = face_landmarks
                                tmpList = []
                                for idx,lm in enumerate(mypose.landmark):
                                    point=np.array([lm.x, lm.y, lm.z])
                                    tmpList.append(point)

                            ear = eye_aspect_ratio(image, tmpList, RIGHT_EYE, LEFT_EYE)
        #                     cv2.putText(image, "Open eyes 5 seconds", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, red, 3)
                            print(ear, totalTime)
                            open_ear.append(ear)

                            cv2.imshow("Open eyes 5 seconds", image)

                            t = time.time()
                            currTime = t - prevTime
                            prevTime = t

                            totalTime += currTime
                        if totalTime > 5.0:
                            print("end")
                            break
                    break                

cap.release()
cv2.destroyAllWindows()
# cap.release()
# cv2.destroyAllWindows()

In [ ]:
close_ear = []

drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
        while cap.isOpened():
            success, image = cap.read()

            if not success:
                print("웹캠 인식 불가")
                continue

            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # image로부터 랜드마크 반환
            results = face_mesh.process(image)
            
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            cv2.putText(image, "enter q and close eyes 5 sec", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, red, 3)
            cv2.imshow('hello', image)
            
            if cv2.waitKey(5) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
            # 반환된 랜드마크를 image에 덧씌움
                with mp_face_mesh.FaceMesh(
                max_num_faces=1,
                refine_landmarks=True,
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5) as face_mesh:
                    prevTime = time.time()
                    totalTime = 0
                    currTime = 0
                    while cap.isOpened():
                        success, image = cap.read()
                        image.flags.writeable = False
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                        # image로부터 랜드마크 반환
                        results = face_mesh.process(image)

                        image.flags.writeable = True
                        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                        if results.multi_face_landmarks:
                            for face_landmarks in results.multi_face_landmarks:
                                mypose = face_landmarks
                                tmpList = []
                                for idx,lm in enumerate(mypose.landmark):
                                    point=np.array([lm.x, lm.y, lm.z])
                                    tmpList.append(point)

                            ear = eye_aspect_ratio(image, tmpList, RIGHT_EYE, LEFT_EYE)
        #                     cv2.putText(image, "Open eyes 5 seconds", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, red, 3)
                            print(ear, totalTime)
                            close_ear.append(ear)

                            cv2.imshow("enter q and close eyes 5 sec", image)

                            t = time.time()
                            currTime = t - prevTime
                            prevTime = t

                            totalTime += currTime
                        if totalTime > 5.0:
                            print("end")
                            break
                    break                

cap.release()
cv2.destroyAllWindows()